In [1]:
import pandas as pd

import geopandas as gpd
from glob import glob

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
kdprov = '94'

In [3]:
# open data sp
data_peta_w_ruta = gpd.read_file("data/output/0_raster_data_desa/desa_pend_rasterize_"+kdprov+".gpkg").to_crs('ESRI:54009')#.area
data_peta_w_ruta['idkab']=data_peta_w_ruta.iddesa.apply(lambda y: str(y)[:4])


In [4]:
data_peta_w_ruta.head(4)

,iddesa,DN,id,pop2020,geometry,idkab
0,9401010006,0.0,0.0,422.0,"MULTIPOLYGON (((13825900.000 -1004900.000, 138...",9401
1,9401010007,1.0,1.0,659.0,"MULTIPOLYGON (((13833000.000 -1028000.000, 138...",9401
2,9401010008,2.0,2.0,363.0,"POLYGON ((13831400.000 -987500.000, 13831500.0...",9401
3,9401010011,3.0,3.0,377.0,"MULTIPOLYGON (((13819900.000 -1005600.000, 138...",9401


In [5]:
data_builtup_kabkota=glob("data/input/GHSL Data/2020/gpkg/ghs_built_"+kdprov+"*_nonzero.shp")

In [6]:
data_builtup_kabkota

['data/input/GHSL Data/2020/gpkg/ghs_built_9429_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9401_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9434_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9412_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9414_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9427_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9417_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9418_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9431_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9409_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9420_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9413_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9403_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9402_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9471_nonzero.shp',
 'data/input/GHSL Data/2020/gpkg/ghs_built_9433_nonzero.shp',
 'data/i

In [7]:
import os.path
def pop_dens(dt):
    global data_peta_w_ruta
    kdkab=dt.split('/')[5].replace('ghs_built_','').replace('_nonzero.shp','')
    if ((os.path.exists("data/output/1_population_density/grid_pop_"+kdkab+"_100m.gpkg")==False) 
        or (os.path.exists("data/output/1_population_density/grid_pop_"+kdkab+"_100m.gpkg-journal")==True)):
        data_builtup=gpd.read_file(dt)#.to_crs(('ESRI:54009'))
        data_builtup['DN']=data_builtup.value
        data_peta_w_ruta.pop2020.sum()
        data_builtup['centroid'] = data_builtup.centroid
        data_builtup['polygon'] = data_builtup['geometry']
        data_builtup.crs='ESRI:54009'
        data_peta_w_ruta.crs='ESRI:54009'
        data_builtup = data_builtup.loc[:,['DN','centroid','polygon']].set_geometry("centroid").sjoin(data_peta_w_ruta)
        data_builtup['TOTAL_PEND']=data_builtup['pop2020']
    # data_builtup = data_builtup.loc[:,['DN','geometry']].sjoin(data_peta_w_ruta)
        data_builtup['p_area'] = data_builtup['DN_left'] / data_builtup.groupby('iddesa').DN_left.transform('sum')
        data_builtup['j_pend'] = list(data_builtup['p_area']*data_builtup['TOTAL_PEND'])
    # data_builtup[['DN_left', 'geometry', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].to_file("data/input/grid_pop_"+kdprov+"_100m.gpkg", driver="GPKG")
        data_builtup[['DN_left', 'centroid', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].to_file("data/temp/grid_pop_"+kdkab+"_100m.gpkg", driver="GPKG")
        data_=data_builtup[['DN_left', 'polygon', 'iddesa','TOTAL_PEND', 'p_area','j_pend']].set_geometry("polygon")
        data_.crs='ESRI:54009'
        data_.to_file("data/output/1_population_density/grid_pop_"+kdkab+"_100m.gpkg", driver="GPKG")
    else:
        print('SKIP for: ',kdkab)

In [8]:
from tqdm.contrib.concurrent import process_map
process_map(pop_dens,data_builtup_kabkota)   

  0%|          | 0/29 [00:00<?, ?it/s]

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometr

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: ESRI:54009

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometr

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]